# Typed Objects

In addition to the interfaces offered by `ObjectAPIClient` and `DownloadedObject`, which provides access to Geoscience Objects in a way that is agnostic to the specific object type, the `evo-objects` package also provides a set of "typed objects" that represent specific object types.

As usual, we need to first authenticate, which we can do using the `ServiceManagerWidget` from the `evo-notebooks` package

In [1]:
from evo.notebooks import ServiceManagerWidget

manager = await ServiceManagerWidget.with_auth_code(
    client_id="core-compute-tasks-notebooks",
    base_uri="https://qa-ims.bentley.com",
    discovery_url="https://int-discover.test.api.seequent.com",
).login()

ServiceManagerWidget(children=(VBox(children=(HBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR…

## Creating a regular 3D grid object

To create a new 'regular-3d-grid' object, we use the `Regular3DGridData` class to define all of the properties of the grid, then pass that to the `Regular3DGrid.create` class method.

The `cell_data` and `vertex_data` properties are dataframes that define the attributes of the cells and vertices respectively. Each column in the dataframe represents a different attribute.

Note the order of the data is important, and must match the order of the cells and vertices in the grid. The cells are ordered such that the x-axis varies the fastest, then y, then z. So for a grid of size (10, 10, 5), the first 10 rows of the dataframe correspond to the cells at (0, 0, 0) to (9, 0, 0), the next 10 rows correspond to (0, 1, 0) to (9, 1, 0), and so on.

In [4]:
import numpy as np
import pandas as pd

from evo.objects.typed import Point3, Regular3DGrid, Regular3DGridData, Rotation, Size3d, Size3i

grid = Regular3DGridData(
    name="Test Grid21",
    origin=Point3(0, 0, 0),
    size=Size3i(10, 10, 5),
    cell_size=Size3d(2.5, 5, 5),
    cell_data=pd.DataFrame(
        {
            "value": np.random.rand(10 * 10 * 5),
            "cat": pd.Categorical.from_codes(np.random.choice(range(4), size=10 * 10 * 5), ["a", "b", "c", "d"]),
        }
    ),
    vertex_data=pd.DataFrame(
        {
            "elevation": np.random.rand(11 * 11 * 6),
        }
    ),
    rotation=Rotation(90, 0, 0),
)
created_grid = await Regular3DGrid.create(manager, grid)
print(f"Created regular-3d-grid object: {created_grid.metadata.url}")

Created regular-3d-grid object: https://350mt.api.integration.seequent.com/geoscience-object/orgs/829e6621-0ab6-4d7d-96bb-2bb5b407a5fe/workspaces/783b6eef-01b9-42a7-aaf4-35e153e6fcbe/objects/ad86e620-0fc5-446f-b553-86463c762e86?version=1767906778051410205


## Downloading a regular 3D grid object

To download an existing 'regular-3d-grid' object, an inspect it, you can use the `Regular3DGrid.from_reference` class method. This method takes the context and the URL of the object to download.

For the example, we will download the object we just created.

In [6]:
grid = await Regular3DGrid.from_reference(manager, created_grid.metadata.url)

print(f"Downloaded regular-3d-grid object with name: {grid.name}")
print(f"Origin: {grid.origin}")
print(f"Size: {grid.size}")
print(f"Cell Size: {grid.cell_size}")
print(f"Rotation: {grid.rotation}")
print(f"Bounding Box: {grid.bounding_box}")

Downloaded regular-3d-grid object with name: Test Grid21
Origin: Point3(x=0.0, y=0.0, z=0.0)
Size: Size3i(nx=10, ny=10, nz=5)
Cell Size: Size3d(dx=2.5, dy=5.0, dz=5.0)
Rotation: Rotation(dip_azimuth=90.0, dip=0.0, pitch=0.0)
Bounding Box: BoundingBox(min_x=0.0, min_y=-25.0, max_x=50.0, max_y=3.061616997868383e-15, min_z=0.0, max_z=25.0)


To inspect the data, `grid.cells.as_dataframe()`, and `grid.vertices.as_dataframe()` can be used to get the cell and vertex data as pandas DataFrames.

In [7]:
await grid.cells.as_dataframe()

,value,cat
0,0.848008,c
1,0.489342,d
2,0.880559,d
3,0.067766,d
4,0.188070,c
...,...,...
495,0.098823,b
496,0.283614,b
497,0.318200,d
498,0.413474,d


In [8]:
await grid.vertices.as_dataframe()

,elevation
0,0.289931
1,0.179575
2,0.293128
3,0.248778
4,0.074564
...,...
721,0.484114
722,0.730475
723,0.030539
724,0.140452


# Updating a regular 3D grid object

To update an existing 'regular-3d-grid' object, we can modify the properties of the `Regular3DGrid` instance, and then call the `update` method to submit the changes to the Geoscience Object service. This will create a new version of the object with the updated properties.

Updating the cell or vertex data can be done by calling the `set_dataframe` method on the `cells` or `vertices` attributes respectively, this will upload the new data to the Geoscience Object service, but the object will remain the same version until the `update` method is called.

In [ ]:
print(f"Version before update: {grid.metadata.version_id}")

grid.name = "Updated Test Grid"

await grid.cells.set_dataframe(
    pd.DataFrame(
        {
            "value": np.ones(10 * 10 * 5),
        }
    )
)

# Update the object to create a new version with the modified properties
await grid.update()
print(f"Updated version: {grid.metadata.version_id}")

## Working with Block Models

Block models are stored in the Block Model Service, which is separate from the Geoscience Object Service. The `BlockModel` class provides a unified interface that handles both services - it creates the block model in the Block Model Service and automatically creates a corresponding Geoscience Object reference.

### Creating a Regular Block Model

To create a new block model, use `BlockModel.create_regular()` with a `RegularBlockModelData` object:

> **Note:** If the imports below fail, you may need to add the local source packages to your Python path. Run the cell below first.

In [2]:
# Setup for local development source - run this cell FIRST before any other imports
# This cell should only be run ONCE per kernel session
import sys

# Remove any cached evo modules to force reimport from local source
# Must also clear evo.common to reset the CustomTypedError registry
mods_to_remove = [key for key in list(sys.modules.keys()) if key.startswith('evo')]
for mod in mods_to_remove:
    del sys.modules[mod]

# Add local source paths at the beginning (if not already present)
local_paths = [
    r"C:\Source\evo-python-sdk\packages\evo-blockmodels\src",
    r"C:\Source\evo-python-sdk\packages\evo-objects\src",
    r"C:\Source\evo-python-sdk\packages\evo-sdk-common\src",
]
for path in local_paths:
    if path not in sys.path:
        sys.path.insert(0, path)

print("Local source paths configured - restart kernel if you see TYPE_ID errors")

Local source paths configured - restart kernel if you see TYPE_ID errors


In [4]:
import numpy as np
import pandas as pd
from evo.objects.typed import BlockModel, RegularBlockModelData, Point3, Size3i, Size3d
from evo.blockmodels import Units

# Create sample block model data
# Define the grid parameters
origin = (0, 0, 0)
n_blocks = (10, 10, 5)
block_size = (2.5, 5.0, 5.0)
total_blocks = n_blocks[0] * n_blocks[1] * n_blocks[2]

# Generate block centroid coordinates (x, y, z)
# Centroids are at the center of each block
centroids = []
for k in range(n_blocks[2]):
    for j in range(n_blocks[1]):
        for i in range(n_blocks[0]):
            # Calculate centroid position: origin + (index + 0.5) * block_size
            x = origin[0] + (i + 0.5) * block_size[0]
            y = origin[1] + (j + 0.5) * block_size[1]
            z = origin[2] + (k + 0.5) * block_size[2]
            centroids.append((x, y, z))

# Create DataFrame with x, y, z coordinates (more user-friendly than i, j, k)
block_data = pd.DataFrame({
    "x": [c[0] for c in centroids],
    "y": [c[1] for c in centroids],
    "z": [c[2] for c in centroids],
    "grade": np.random.rand(total_blocks) * 10,  # Random grade values 0-10
    "density": np.random.rand(total_blocks) * 2 + 2,  # Random density 2-4
})

# Create the block model using BlockModel.create_regular()
# Use the Units class for valid unit IDs
bm_data = RegularBlockModelData(
    name="Example Block Model 12",
    description="A sample block model created from the notebook",
    origin=Point3(x=origin[0], y=origin[1], z=origin[2]),
    n_blocks=Size3i(nx=n_blocks[0], ny=n_blocks[1], nz=n_blocks[2]),
    block_size=Size3d(dx=block_size[0], dy=block_size[1], dz=block_size[2]),
    cell_data=block_data,
    crs="EPSG:28354",
    size_unit_id=Units.METRES,
    units={"grade": Units.GRAMS_PER_TONNE, "density": Units.TONNES_PER_CUBIC_METRE},
)

# Create the block model - this handles both Block Model Service and Geoscience Object creation
block_model = await BlockModel.create_regular(manager, bm_data)

print(f"Created block model: {block_model.name}")
print(f"Block Model UUID: {block_model.block_model_uuid}")
print(f"Geometry type: {block_model.geometry.model_type}")
print(f"Origin: {block_model.geometry.origin}")
print(f"N blocks: {block_model.geometry.n_blocks}")
print(f"Block size: {block_model.geometry.block_size}")
print(f"Attributes: {[attr.name for attr in block_model.attributes]}")

Created block model: Example Block Model 12
Block Model UUID: 3ee13206-ad34-45f4-b95c-5732e68dff41
Geometry type: regular
Origin: Point3(x=0.0, y=0.0, z=0.0)
N blocks: Size3i(nx=10, ny=10, nz=5)
Block size: Size3d(dx=2.5, dy=5.0, dz=5.0)
Attributes: ['grade', 'density']


### Loading an Existing Block Model

You can load an existing `BlockModel` using `from_reference`, just like other typed objects:

In [5]:
# Load the block model we just created using its URL
loaded_bm = await BlockModel.from_reference(manager, block_model.metadata.url)

print(f"Loaded BlockModel: {loaded_bm.name}")
print(f"Block Model UUID: {loaded_bm.block_model_uuid}")
print(f"Bounding Box: {loaded_bm.bounding_box}")

Loaded BlockModel: Example Block Model 12
Block Model UUID: 3ee13206-ad34-45f4-b95c-5732e68dff41
Bounding Box: BoundingBox(min_x=0.0, min_y=0.0, max_x=25.0, max_y=50.0, min_z=0.0, max_z=25.0)


### Accessing Block Model Data

The `BlockModel` provides methods to access the actual block data from the Block Model Service:

In [6]:
# Get all block data as a DataFrame
df = await block_model.get_data(columns=["*"])
print(f"Retrieved {len(df)} blocks")
df.head(10)

Retrieved 500 blocks


,x,y,z,density,grade
0,1.25,2.5,2.5,2.480837,0.008433
1,1.25,2.5,7.5,2.007310,2.295891
2,1.25,7.5,2.5,3.683386,9.374861
3,1.25,7.5,7.5,3.860459,5.374490
4,1.25,12.5,2.5,3.904405,7.347877
5,1.25,12.5,7.5,2.973604,7.474377
6,3.75,2.5,2.5,2.617545,2.289998
7,3.75,2.5,7.5,2.016279,2.412427
8,3.75,7.5,2.5,2.395363,5.541266
9,3.75,7.5,7.5,2.463445,4.276713


In [7]:
# You can also query specific columns
grade_data = await block_model.get_data(columns=["grade"])
print(f"Grade statistics:")
print(grade_data["grade"].describe())

Grade statistics:
count    500.000000
mean       5.157045
std        2.850108
min        0.004934
25%        2.801187
50%        5.240961
75%        7.469605
max        9.995830
Name: grade, dtype: float64


### Adding New Attributes to a Block Model

You can add new attributes to the block model. This creates a new version in the Block Model Service:

In [8]:
# Calculate a new attribute based on existing data
# Get current data including coordinates
df = await block_model.get_data(columns=["x", "y", "z", "grade", "density"])

# Create new attribute DataFrame with x, y, z coordinates
df_with_new_attr = pd.DataFrame({
    "x": df["x"],
    "y": df["y"],
    "z": df["z"],
    "metal_content": df["grade"] * df["density"],  # grade * density
})

# Add the new attribute to the block model
# Use Units class for valid unit IDs
new_version = await block_model.add_attribute(
    df_with_new_attr,
    attribute_name="metal_content",
    unit=Units.KG_PER_CUBIC_METRE,
)

print(f"Added attribute 'metal_content', new version: {new_version.version_id}")

Added attribute 'metal_content', new version: 3


In [10]:
new_version

Version(bm_uuid=UUID('3ee13206-ad34-45f4-b95c-5732e68dff41'), version_id=3, version_uuid=UUID('330408ff-4a84-4a80-91d9-b237b40beaa5'), parent_version_id=2, base_version_id=2, geoscience_version_id='1768422434602050397', created_at=datetime.datetime(2026, 1, 14, 20, 27, 14, 136105, tzinfo=TzInfo(0)), created_by=ServiceUser(id=UUID('279fddd2-7d0f-422f-8a7e-3e9c42a0f3af'), name='Denis Simo', email='Denis.Simo@bentley.com'), comment='', bbox=BBox(i_minmax=IntRange(max=9, min=0), j_minmax=IntRange(max=9, min=0), k_minmax=IntRange(max=4, min=0)), columns=[Column(col_id='i', data_type=<DataType.UInt32: 'UInt32'>, title='i', unit_id=None), Column(col_id='j', data_type=<DataType.UInt32: 'UInt32'>, title='j', unit_id=None), Column(col_id='k', data_type=<DataType.UInt32: 'UInt32'>, title='k', unit_id=None), Column(col_id='version_id', data_type=<DataType.UInt32: 'UInt32'>, title='version_id', unit_id=None), Column(col_id='sub_block_derivation', data_type=<DataType.Utf8: 'Utf8'>, title='sub_block_

### Updating Multiple Attributes

For more complex updates (adding multiple columns, updating existing columns, or deleting columns), use `update_attributes`:

In [24]:
# Create data with multiple new attributes using x, y, z coordinates
df_updates = pd.DataFrame({
    "x": df["x"],
    "y": df["y"],
    "z": df["z"],
    "classification": np.where(df["grade"] > 5, "high", "low"),
    "value_index": df["grade"] * df["density"] * 100,
})

# Add multiple new columns at once
version = await block_model.update_attributes(
    df_updates,
    new_columns=["classification", "value_index"],
)

print(f"Updated block model, new version: {version.version_id}")

# Verify the new attributes
updated_df = await block_model.get_data(columns=["*"])
print(f"Columns now available: {list(updated_df.columns)}")


delete=[] new=[ColumnLite(data_type=<DataType.Utf8: 'Utf8'>, title='classification', unit_id=None), ColumnLite(data_type=<DataType.Float64: 'Float64'>, title='value_index', unit_id=None)] rename=[] update=[] update_metadata=None
Updated block model, new version: 4
Columns now available: ['x', 'y', 'z', 'density', 'grade']
